In [ ]:
import numpy as np
from numpy.random import default_rng

import matplotlib.pyplot as plt
import matplotlib.lines as lines
import matplotlib as mpl
import scipy.spatial.distance as dist

import IBloFunMatch_inter as ibfm
output_dir = "output"

This notebook illustrates the difficulty with matching stability that might come up with nesting behavour of bars.
First, sample a pair of circles and take a sample from them.

In [ ]:
rng = default_rng(2)
r = 10
radius = rng.uniform(r-0.4*r, r+0.4*r, 100)
angle = np.pi * rng.uniform(0, 2, 100)
r=8
C1 = np.vstack((np.sqrt(radius) * np.cos(angle), np.sqrt(radius) * np.sin(angle))).transpose()
radius = rng.uniform(r-0.4*r, r+0.4*r, 80)
angle = np.pi * rng.uniform(0, 2, 80)
C2 = np.vstack((np.sqrt(radius) * np.cos(angle), np.sqrt(radius) * np.sin(angle))).transpose()+[0,6.5]
Y=np.vstack((C1,C2))

In [ ]:
subset_indices = rng.choice(range(len(Y)), replace=False, size=110)
center_pts_bool = np.sqrt(np.sum((Y-[0,3.5])**2, axis=1))<1.4
subset_indices = [idx for idx in subset_indices if not center_pts_bool[idx]]
X = Y[subset_indices]

In [ ]:
%%capture
Dist_X = dist.squareform(dist.pdist(X))
Dist_Y = dist.squareform(dist.pdist(Y))
IBloFunMatch_o = ibfm.get_IBloFunMatch_subset(Dist_X, Dist_Y, subset_indices, output_dir)

Notice that the induced matching and the induced block functions are different in dimension 1

In [ ]:
IBloFunMatch_o["induced_matching_1"]

In [ ]:
IBloFunMatch_o["block_function_1"]

In [ ]:
IBloFunMatch_o["pm_matrix_1"]

In [ ]:
fig, ax = plt.subplots(ncols=2, nrows=2, figsize=(6,6))
ax[1,0].scatter(X[:,0], X[:,1], c="orange")
ax[1,1].scatter(Y[:,0], Y[:,1], c="navy")
ax[1,0].set_title("Subset")
ax[1,1].set_title("Dataset")
ibfm.plot_matching(IBloFunMatch_o, output_dir, ax[0], fig)
fig.suptitle("Original point cloud and sample")
plt.savefig("plots/stability/two_circles.png")

Loop through noisy samples of the same pair.

In [ ]:
%%capture
rng = default_rng(10)
for idx in range(10):
    eps = 0.1*(1+idx)
    Y2 = Y + rng.normal(size=Y.shape)*eps
    X2 = Y2[subset_indices]
    Dist_X2 = dist.squareform(dist.pdist(X2))
    Dist_Y2 = dist.squareform(dist.pdist(Y2))
    IBloFunMatch_o2 = ibfm.get_IBloFunMatch_subset(Dist_X2, Dist_Y2, subset_indices, output_dir)
    fig, ax = plt.subplots(ncols=2, nrows=2, figsize=(6,6))
    ax[1,0].scatter(X2[:,0], X2[:,1], c="orange")
    ax[1,1].scatter(Y2[:,0], Y2[:,1], c="navy")
    ax[1,0].set_title("Subset")
    ax[1,1].set_title("Dataset")
    ibfm.plot_matching(IBloFunMatch_o2, output_dir, ax[0], fig)
    fig.suptitle(f"Noise: {eps:.2f}")
    plt.savefig(f"plots/stability/two_circles{idx}.png")

In [ ]:
%%capture
rng = default_rng(10)
eps = 0.4
for idx in range(10):
    Y2 = Y + rng.normal(size=Y.shape)*eps
    X2 = Y2[subset_indices]
    output_dir="output/"
    Dist_X2 = dist.squareform(dist.pdist(X2))
    Dist_Y2 = dist.squareform(dist.pdist(Y2))
    IBloFunMatch_o2 = ibfm.get_IBloFunMatch_subset(Dist_X2, Dist_Y2, subset_indices, output_dir)
    fig, ax = plt.subplots(ncols=2, nrows=2, figsize=(6,6))
    ax[1,0].scatter(X2[:,0], X2[:,1], c="orange")
    ax[1,1].scatter(Y2[:,0], Y2[:,1], c="navy")
    ax[1,0].set_title("Subset")
    ax[1,1].set_title("Dataset")
    ibfm.plot_matching(IBloFunMatch_o2, output_dir, ax[0], fig)
    fig.suptitle(f"Noise: {eps:.2f}, iteration= {idx}")
    plt.savefig(f"plots/stability/two_circ_noise_const_{idx}.png")

## Perfect Circles Example

In [ ]:
r = 3
num_big = 30
num_small=15
angle = np.linspace(0,2*np.pi,num_big)
C1 = np.vstack((r * np.cos(angle), r * np.sin(angle))).transpose()
angle = np.linspace(0,2*np.pi,num_small)
C2 = np.vstack((0.7*r * np.cos(angle), 0.7*r * np.sin(angle))).transpose()+[0,3]
Y=np.vstack((C1,C2))
# Take subset sample
rng = default_rng(5)
subset_indices = list(range(0,num_big,1))+list(range(num_big,num_big+num_small, 2))
center_pts_bool = np.sqrt(np.sum((Y-[0,3])**2, axis=1))<2
subset_indices = [idx for idx in subset_indices if not center_pts_bool[idx]]
X = Y[subset_indices]

In [ ]:
len(subset_indices)

In [ ]:
len(np.unique(subset_indices))

In [ ]:
%%capture
Dist_X = dist.squareform(dist.pdist(X))
Dist_Y = dist.squareform(dist.pdist(Y))
IBloFunMatch_o = ibfm.get_IBloFunMatch_subset(Dist_X, Dist_Y, subset_indices, output_dir)

In [ ]:
IBloFunMatch_o["pm_matrix_1"]

In [ ]:
assert(IBloFunMatch_o["induced_matching_1"]==IBloFunMatch_o["block_function_1"])

In [ ]:
# Plot POints 
fig, ax = plt.subplots(ncols=2, nrows=2, figsize=(7,7))
ax[1,0].scatter(X[:,0], X[:,1], c="orange")
ax[1,1].scatter(Y[:,0], Y[:,1], c="navy")
ax[1,0].set_title("Subset")
ax[1,1].set_title("Dataset")
ibfm.plot_matching(IBloFunMatch_o, output_dir, ax[0], fig)
plt.savefig("plots/perfect_circles/two_circles_perfect.png")

In [ ]:
%%capture
rng = default_rng(10)
for i in range(5):
    eps = 0.1 + 0.1*i
    Y2 = Y + rng.normal(size=Y.shape)*eps
    X2 = Y2[subset_indices]
    fig, ax = plt.subplots(ncols=2, nrows=2, figsize=(6,6))
    ax[1,0].scatter(X2[:,0], X2[:,1], c="orange")
    ax[1,1].scatter(Y2[:,0], Y2[:,1], c="navy")
    ax[1,0].set_title("Subset")
    ax[1,1].set_title("Dataset")
    output_dir="output/"
    Dist_X2 = dist.squareform(dist.pdist(X2))
    Dist_Y2 = dist.squareform(dist.pdist(Y2))
    IBloFunMatch_o = ibfm.get_IBloFunMatch_subset(Dist_X2, Dist_Y2, subset_indices, output_dir)
    assert(IBloFunMatch_o["induced_matching_1"]==IBloFunMatch_o["block_function_1"])
    ibfm.plot_matching(IBloFunMatch_o, output_dir, ax[0], fig)
    fig.suptitle(f"Noise: {eps:.2f}")
    plt.savefig(f"plots/perfect_circles/two_circles_perfect_{i}.png")

In [ ]:
rng = default_rng(10)
eps=0.3
for i in range(8):
    Y2 = Y + rng.normal(size=Y.shape)*eps
    X2 = Y2[subset_indices]
    fig, ax = plt.subplots(ncols=2, nrows=2, figsize=(6,6))
    ax[1,0].scatter(X2[:,0], X2[:,1], c="orange")
    ax[1,1].scatter(Y2[:,0], Y2[:,1], c="navy")
    ax[1,0].set_title("Subset")
    ax[1,1].set_title("Dataset")
    output_dir="output/"
    Dist_X2 = dist.squareform(dist.pdist(X2))
    Dist_Y2 = dist.squareform(dist.pdist(Y2))
    IBloFunMatch_o = ibfm.get_IBloFunMatch_subset(Dist_X2, Dist_Y2, subset_indices, output_dir)
    ibfm.plot_matching(IBloFunMatch_o, output_dir, ax[0], fig)
    fig.suptitle(f"Noise: {eps:.2f}, iteration: {i}")
    plt.savefig(f"plots/perfect_circles/two_circ_const_noise_{i}.png")